In [ ]:
!wget -q https://phonetics-spbu.github.io/courses/python_textbook/files/cta0001.wav
!wget -q https://phonetics-spbu.github.io/courses/python_textbook/files/cta0001_stereo.wav
!wget -q https://phonetics-spbu.github.io/courses/python_textbook/files/cta0001.sbl
!wget -q https://phonetics-spbu.github.io/courses/python_textbook/files/24bit.wav


In [ ]:
%pip install wavio

Для хранения звуковых данных существует большое количество различных форматов. Они делятся на две категории: те, в которых данные сжимаются, и те, в которых этого не происходит. Сжатие может происходить как с потерями (в этом случае невозможно с точностью восстановить исходные данные), так и без них.

### Файлы WAV

Самый распространённый формат для хранения звуковых данных без сжатия &mdash; WAV. В нём хранятся значения амплитуды каждого отсчёта цифрового аудиосигнала. Формат WAV является разновидностью формата RIFF (Resource Interchange File Format), который подразумевает хранение данных блоками (англ. _chunks_). Файлы формата WAV состоят из двух основных частей: заголовка и собственно данных. Структура заголовка приведена в таблице ниже:

|Название поля|Количество байт|Информация, содержащаяся в поле|
|---|---|---|
|FileTypeBlocID|4|Строка `"RIFF"`: идентификатор формата|
|FileSize|4|Размер файла в байтах минус 8|
|FileFormatID|4|Строка `"WAVE"`: идентификатор формата|
|FormatBlocID|4|Строка `"fmt "`: идентификатор чанка (описание формата)|
|BlocSize|4|Размер текущего блока минус 8|
|AudioFormat|2|Формат аудио (см. ниже)|
|NbrChannels|2|Количество каналов (1 &mdash; моно, 2 &mdash; стерео)|
|Frequency|4|Частота дискретизации (в герцах)|
|BytePerSec|4|Количество байт в секунду|
|BytePerBloc|2|Количество байт на один блок|
|BitsPerSample|2|Количество бит на один отсчёт|
|DataBlocID|4|Строка `"data"`: идентификатор чанка (сами данные)|
|DataSize|4|Размер данных в байтах|
|SampledData|(вариативно)|Данные об амплитуде каждого отсчёта|

В заголовке может храниться и другая информация, например, метаданные. Если в файле больше одного канала, то данные об амплитуде отсчётов из каждого канала чередуются. Например, если в файле два канала, то в поле SampledData хранится сначала первый отсчёт первого канала, затем первый отсчёт второго канала, затем второй отсчёт первого канала и т.&nbsp;д.

#### Основные стандарты кодирования аудиоданных

В поле AudioFormat хранится специальное число, которое показывает, в каком виде хранятся данные об амплитуде каждого отсчёта. Используются следующие основные стандарты:

1. Импульсно-кодовая модуляция (_pulse code modulation_, _PCM_): значения амплитуды хранятся напрямую. Они могут храниться в виде целых чисел (код 1, 0x0001) или в виде чисел с плавающей запятой по стандарту IEEE 754 (код 3, 0x0003). Целочисленный PCM (_integer PCM_) используется в подавляющем большинстве случаев, и далее будет предполагаться, что используется он, если не сказано иное.

2. Адаптивная дифференциальная импульсно-кодовая манипуляция (_adaptive differential pulse code modulation_, _ADPCM_): хранятся не значения амплитуды, а разности амплитуд соседних отсчётов, при этом шаг квантования варьируется (код 2, 0x0002).

3. Мю-закон (_mu-law_): амплитуда кодируется по специальной формуле (код 7, 0x0007).

$$ F(x) = \text{sgn}(x)  \frac{\ln(1 + \mu \lvert x \rvert)}{\ln(1 + \mu)}, -1 \le x \le 1, $$

где &mu; = 255.

4. А-закон (_A-law_): амплитуда кодируется по специальной формуле (код 6, 0x0006).

$$ F(X) = \text{sgn}(x) \begin{cases} \frac{A \lvert x \rvert}{1 + \ln(A)}, & \lvert x \rvert \lt \frac{1}{A}, \\ \frac{1 + \ln(A \lvert x \rvert)}{1 + \ln(A)}, & \frac{1}{A} \le \lvert x \rvert \lt 1, \end{cases} $$

где A = 87.6 в европейском стандарте.

#### Количество байт на отсчёт

Количество байт, отведённое на хранение одного отсчёта, будет определять количество уровней квантования, а тип данных, избранный для хранения, будет определять диапазон значений амплитуды (см. таблицу ниже).

|Кол-во байт|Тип данных в языке C|Диапазон значений|
|---|---|---|
|1|char|0&mdash;255|
|2|short|&minus;32768&mdash;32767|
|3|&mdash;|&mdash;|
|4|int|&minus;2147483648&mdash;2147483647|

Обратите внимание, что в 8-битных файлах значения амплитуды хранятся в беззнаковом типе, поэтому отрицательных значений в таких файлах быть не может.

Несмотря на то, что существует стандарт записи, при котором на один отсчёт отводится 3 байта (24 бита), 24-битного целочисленного типа не существует. В связи с этим такие файлы поддерживаются не всеми библиотеками.

Если значения хранятся в виде чисел с плавающей запятой, то на один отсчёт может отводиться 32 или 64 бита (см. таблицу ниже).

|Кол-во байт|Тип данных в языке C|Диапазон|
|---|---|---|
|4|float|&minus;3.40282347&times;10<sup>38</sup>&mdash;&minus;1.17549435&times;10<sup>&minus;38</sup>, 0, 1.17549435&times;10<sup>&minus;38</sup>&mdash;3.40282347&times;10<sup>38</sup>|
|8|double|&minus;1.797693134862315&times;10<sup>308</sup>&mdash;&minus;2.225073858507201&times;10<sup>&minus;308</sup>, 0, 2.225073858507201&times;10<sup>&minus;308</sup>&mdash;1.797693134862315&times;10<sup>308</sup>|

#### Нормализация амплитуд

В связи с большим разбросом диапазона амплитуд, при обработке их принято нормализовывать путём деления на максимальное возможное значение для данного типа данных. Таким образом, все амплитуды приводятся в диапазон от &minus;1 до 1. В программе Wave Assistant такая нормализация не проводится, и по оси ординат отложена амплитуда в изначальных, &laquo;сырых&raquo; значениях. В программе Praat, напротив, такая нормализация проводится, и по оси ординат отложена амплитуда от &minus;1 до 1.

#### Положение отсчётов

В формате WAV позиция каждого отсчёта не кодируется, все отсчёты идут просто по порядку. Поэтому, чтобы определить, в каком месте сигнала находится тот или иной отсчёт, необходимо знать номер этого отсчёта и частоту дискретизации, которая показывает, сколько отсчётов помещается в одну секунду. Для перевода из отсчётов в секунды можно воспользоваться формулой:

$$ t = \frac{n}{F_s}, $$

где F<sub>s</sub> &mdash; частота дискретизации.

Обратная формула (поскольку номер отсчёта &mdash; целое число, необходимо округлить результат):

$$ n = \left[ t \cdot F_s \right] $$

#### Файлы без заголовка

Иногда, если информация о формате файла (стандарте кодирования, частоте дискретизации и др.) уже известна, заголовок у файла не пишется. Такие файлы называются RAW (&laquo;сырые&raquo; файлы) и могут иметь расширения .raw, .pcm или .sbl (этим расширением пользуется программа Wave Assistant).

![Рис. 1. Файлы с заголовком и без заголовка](https://phonetics-spbu.github.io/courses/python_textbook/images/headless.jpg)

### Работа с файлами WAV с помощью встроенных библиотек Python

Для работы с файлами WAV нам понадобится встроенная библиотека [`wave`](https://docs.python.org/3/library/wave.html).

#### Чтение файлов

Чтобы открыть файл, воспользуемся функцией `wave.open()`:

In [ ]:
import wave

f = wave.open("cta0001.wav", mode="rb")

Вторым аргументом мы передали режим, в котором открываем файл: в данном случае мы открываем его для чтения. Тогда функция вернёт объект класса `Wave_read`, через методы которого мы будем получать доступ к данным файла.

Получим данные о параметрах файла: длине сигнала в отсчётах, частоте дискретизации, количестве байт на один отсчёт и количестве каналов.

In [ ]:
num_samples = f.getnframes()
samplerate = f.getframerate()
sampwidth = f.getsampwidth()
num_channels = f.getnchannels()

Параметры можно получить все сразу, используя метод `.getparams()`. Он вернёт кортеж (`namedtuple`) со следующими параметрами: количество каналов, количество байт на отсчёт, частота дискретизации, количество отсчётов, тип сжатия, название типа сжатия (на самом деле сжатие в Python не поддерживается, поэтому последние два параметра будут всегда `'NONE'` и `'not compressed'`).

Прочитать отсчёты из файла можно с помощью метода `.readframes()`. Он принимает в качестве аргумента количество чисел, которые нужно прочитать. Оно будет равно длине сигнала в отсчётах, умноженной на количество каналов.

In [ ]:
num_frames = num_samples * num_channels
data_bytes = f.readframes(num_frames)

Метод возвращает объект типа `bytes` &mdash; то есть просто последовательность байт. Её необходимо преобразовать в последовательность целых чисел. Для этого воспользуемся методом-конструктором `int.from_bytes()`. Этот метод принимает на вход последовательность байт, соответствующих одному числу, порядок байт в числе и является ли тип знаковым.

Под порядком байт (англ. _endianness_) понимается порядок, в котором записываются байты: сначала может идти старший байт (_most significant byte_), тогда порядок называется _big-endian_ (от старшего к младшему), а может идти младший байт (_least significant byte_), тогда порядок называется _little-endian_ (от младшего к старшему). Первый вариант интуитивно кажется более логичным, потому что он совпадает с правилами записи чисел арабскими цифрами, от старших разрядов к младшим. Однако чаще (в том числе в файлах WAV) используется порядок _little-endian_, поэтому в аргумент `byteorder` мы передадим строку `"little"`.

Как указывалось выше, беззнаковый тип используется только в 8-битных файлах, поэтому в аргумент `signed` передадим выражение, которое будет равно `False` только тогда, когда количество байт на отсчёт (`sampwidth`) не равно 1: `sampwidth != 1`.

In [ ]:
signal = [
    int.from_bytes(
        data_bytes[i:i+sampwidth],
        byteorder="little",
        signed=(sampwidth != 1),
    )
    for i in range(0, len(data_bytes), sampwidth)
]

Теперь в переменной `signal` содержится список из `num_frames` значений, каждое из которых соответствует значению амплитуды одного отсчёта.

#### Библиотека `struct`

Преобразование можно проводить и с помощью библиотеки [`struct`](https://docs.python.org/3/library/struct.html). Для этого воспользуемся функцией `struct.unpack()`. Она принимает в качестве аргументов специальную строку, описывающую формат считываемых чисел, и сами данные.

Форматную строку сконструируем следующим образом: переведём количество чисел в строковый формат и прибавим букву, которая обозначает тип, соответствующий количеству байт на отсчёт (с полной таблицей обозначений можно ознакомиться в [документации](https://docs.python.org/3/library/struct.html#format-characters)).

In [ ]:
import struct

sampwidth_to_char = {1: "c", 2: "h", 4: "i"}
fmt = str(num_frames) + sampwidth_to_char[sampwidth]
signal = struct.unpack(fmt, data_bytes)

Теперь в переменной `signal` содержится кортеж из `num_frames` значений, каждое из которых соответствует значению амплитуды одного отсчёта.

Обратите внимание, что мы нигде специально не указывали порядок байт: `struct` по умолчанию работает с системным порядком байт (а в большинстве случаев это _little-endian_). В том случае, если системный порядок _big-endian_, нужный порядок можно указать перед буквой типа:

In [ ]:
fmt = str(num_frames) + "<" + sampwidth_to_char[sampwidth]

#### Визуализация данных

Воспользуемся библиотекой `matplotlib`, чтобы вывести осциллограмму на экран.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(signal)
plt.xlabel("Time, samples")
plt.ylabel("Amplitude, raw")
plt.show()

![image](https://phonetics-spbu.github.io/courses/python_textbook/images/cta0001_raw.png)

По оси _x_ отложено время, измеренное в отсчётах, а по оси _y_ &mdash; &laquo;сырые&raquo;, т.&nbsp;е. ненормализованные значения амплитуды.

Чтобы по оси _x_ отложить время в секундах, для каждого номера отсчёта определим, какому времени он соответствует. Для этого разделим каждый номер на частоту дискретизации:

In [ ]:
times = [s / samplerate for s in range(len(signal))]
plt.plot(times, signal)
plt.xlabel("Time, seconds")
plt.ylabel("Amplitude, raw")
plt.show()

![image](https://phonetics-spbu.github.io/courses/python_textbook/images/cta0001_seconds.png)

#### Чтение стереофайлов

Попробуем теперь прочитать стереофайл, то есть файл с двумя каналами, по одному для каждого уха. В двухканальных файлах на самом деле можно хранить совсем разные сигналы: например, в диалоговых корпусах часто речь одного собеседника хранится в левом канале, а речь другого &mdash; в правом. Или в одном канале может храниться запись с микрофона, а в другом &mdash; синхронизированная с ней электроглоттограмма.

In [ ]:
f = wave.open("cta0001_stereo.wav", mode="rb")

num_samples = f.getnframes()
samplerate = f.getframerate()
sampwidth = f.getsampwidth()
num_channels = f.getnchannels()

num_frames = num_samples * num_channels
data_bytes = f.readframes(num_frames)

signal = [
    int.from_bytes(
        data_bytes[i:i+sampwidth],
        byteorder="little",
        signed=(sampwidth != 1),
    )
    for i in range(0, len(data_bytes), sampwidth)
]

В файле `cta0001_stereo.wav` левый канал на самом деле идентичен правому. Чтобы в этом убедиться, разделим отсчёты по каналам (как мы помним, отсчёты в файле чередуются). Для этого воспользуемся срезами.

In [ ]:
left = signal[::2]
right = signal[1::2]

Код ниже выведет `True`, что показывает, что каналы действительно идентичны.

In [ ]:
print(left == right)

#### Чтение 24-битных файлов с помощью `struct`

Как указывалось выше, чтение файлов, где на хранение одного отсчёта отводится 3 байта, сталкивается с трудностями. Это связано с тем, что не существует 24-битного целочисленного типа. Такие файлы без проблем прочитаются с помощью `int.from_bytes()`, но при использовании `struct` нам придётся модифицировать прочитанные данные так, чтобы мы могли интерпретировать их как 32-битные целые числа. Прочитаем файл как обычно:

In [ ]:
f = wave.open("24bit.wav")

num_samples = f.getnframes()
samplerate = f.getframerate()
sampwidth = f.getsampwidth() # 3
num_channels = f.getnchannels()
data_bytes = f.readframes(num_samples * num_channels)

Теперь переберём байты тройками и к каждой тройке в начало добавим нулевой байт. Тогда получится 32-битное число, в 2^8^ раз большее имеющегося в виду. Для объединения байт в одну последовательность воспользуемся методом `bytes.join()`, который работает точно так же, как его строковый аналог. В качестве разделителя укажем пустую последовательность байт `b""`.

In [ ]:
data_bytes = b"".join(
    b"".join((b"\x00", data_bytes[i:i+3]))
    for i in range(0, len(data_bytes), 3)
)

Распакуем данные:

In [ ]:
fmt = str(num_samples * num_channels) + "i"
signal = struct.unpack(fmt, data_bytes)

И приведём числа к изначальным величинам путём побитового сдвига (эквивалентно делению на 2^8^).

In [ ]:
signal = [i >> 8 for i in signal]

### Запись файлов

Чтобы записать аудиоданные в файл, нужно преобразовать их в последовательность байт. Для этого можно воспользоваться методом `int.to_bytes()` или библиотекой `struct`.

Для примера сгенерируем синусоидальный сигнал. Зададим частоту дискретизации, количество байт на отсчёт, количество каналов и длительность сигнала в секундах. Чтобы вычислить количество отсчётов в итоговом сигнале, умножим длительность на частоту дискретизации.

In [ ]:
from math import sin, pi

samplerate = 22050
sampwidth = 2
num_channels = 1
duration = 2  # секунды 
num_samples = samplerate * duration

Зададим значение амплитуды нашей синусоиды (половина от максимально возможной). У беззнаковых типов максимальное число определяется как 2<sup>(количество бит на число)</sup> &minus; 1, у знаковых &mdash; 2<sup>(количество бит на число &minus; 1)</sup> &minus; 1.

In [ ]:
max_ampl = 2 ** (sampwidth * 8 - (0 if sampwidth == 1 else 1)) - 1
ampl = 0.5 * max_ampl

Зададим частоту в герцах и сгенерируем синусоиду с помощью функции `math.sin()` и спискового включения. Не забудем сделать так, чтобы все значения сигнала были целыми числами (иначе мы не сможем записать их в файл).

In [ ]:
freq = 100  # герцы
sine = [int(ampl * sin(x * freq * 2 * pi / samplerate)) for x in range(num_samples)]

Изобразим первые 1000 отсчётов синусоиды на графике:

In [ ]:
plt.plot(sine[:1000])
plt.xlabel("Time, samples")
plt.ylabel("Amplitude, raw")
plt.show()

![image](https://phonetics-spbu.github.io/courses/python_textbook/images/sine_1000.png)

Метод `.to_bytes()` есть у каждого целочисленного объекта. Он берёт на вход количество байт на число, порядок записи байт и логическое значение, показывающее, является ли тип знаковым. Объединим полученные последовательности байт в одну большую последовательность при помощи метода `bytes.join()`. В качестве разделителя укажем пустую последовательность байт `b""`.

In [ ]:
data_bytes = b"".join(
    s.to_bytes(
        sampwidth,
        byteorder="little",
        signed=(sampwidth != 1),
    )
    for s in sine
)

То же самое с помощью `struct`:

In [ ]:
sampwidth_to_char = {1: "c", 2: "h", 4: "i"}
fmt = str(num_samples) + sampwidth_to_char[sampwidth]
data_bytes = struct.pack(fmt, *sine)

И теперь последовательность байт можно записать в файл вместе со всеми необходимыми параметрами:

In [ ]:
f = wave.open("sine_wave.wav", "wb")
f.setnchannels(num_channels)
f.setsampwidth(sampwidth)
f.setframerate(samplerate)
f.writeframes(data_bytes)
f.close()

#### Чтение файлов без заголовка

В аудиофайлах без заголовка хранятся только данные, а все параметры должны быть известны заранее, например, из файлов разметки.

Зададим параметры, с которыми будем читать файл:

In [ ]:
samplerate = 22050
sampwidth = 2
num_channels = 1

Откроем файл в режиме бинарного чтения (`"rb"`) с помощью обычной встроенной функции `open()` и считаем оттуда последовательность байт:

In [ ]:
with open("cta0001.sbl", "rb") as f:
    data_bytes = f.read()

И преобразуем её в список целых чисел, как раньше:

In [ ]:
signal = [
    int.from_bytes(
        data_bytes[i:i+sampwidth],
        byteorder="little",
        signed=(sampwidth != 1),
    )
    for i in range(0, len(data_bytes), sampwidth)
]

Если мы хотим воспользоваться `struct`, то сначала вычислим количество отсчётов как количество байт в последовательности (определим с помощью `len()`), делённое на количество байт на один отсчёт:

In [ ]:
sampwidth_to_char = {1: "c", 2: "h", 4: "i"}
num_samples = len(data_bytes) // sampwidth
fmt = str(num_samples) + sampwidth_to_char[sampwidth]
signal = struct.unpack(fmt, data_bytes)

Запись файлов без заголовка осуществляется также с помощью `open()`:

In [ ]:
with open("output.sbl", "wb") as f:
    f.write(data_bytes)

### Работа с файлами WAV с помощью библиотеки `scipy`

Встроенные библиотеки Python предлагают довольно ограниченные возможности для работы с файлами WAV. Так, они поддерживают только целочисленные форматы: при попытке открыть файл с вещественными значениями с помощью `wave.open()` возникнет ошибка. Кроме того, как можно убедиться, чтение аудиофайлов &mdash; не самый простой процесс, требующий нескольких строк кода на открытие каждого файла. Наконец, вычисления в Python достаточно медленные: было бы удобно получать аудиоданные сразу в виде массивов `numpy`.

Для этого можно воспользоваться модулем [`io.wavfile`](https://docs.scipy.org/doc/scipy/reference/io.html#module-scipy.io.wavfile) из библиотеки `scipy`. Именно этот способ является рекомендованным и наиболее удобным на практике.

In [ ]:
from scipy.io import wavfile

В нём определены функции для чтения (`read()`) и записи (`write()`) файлов WAV.

#### Чтение файлов

 Функция `read()` принимает на вход имя файла или файловый объект, открытый для чтения, а возвращает частоту дискретизации (целое число) и `numpy`-массив, содержащий значения амплитуды в каждом отсчёте. Прочитаем файл:

In [ ]:
samplerate, signal = wavfile.read("cta0001.wav")

Длина массива будет соответствовать количеству отсчётов, а тип данных этого массива &mdash; количеству байт на отсчёт. Тип данных хранится в атрибуте `.dtype`:

In [ ]:
print(signal.dtype)

Таблица соответствий стандартов записи и типов `numpy`:

|Стандарт записи|Тип данных|
|---|---|
|8-bit integer|`numpy.uint8`|
|16-bit integer|`numpy.int16`|
|24-bit integer|`numpy.int32`|
|32-bit integer|`numpy.int32`|
|32-bit float|`numpy.float32`|
|64-bit float|`numpy.float64`|

При чтении стереофайлов массив будет двумерным. Посмотрим на размеры каждого измерения, которые хранятся в атрибуте `shape` (это кортеж):

In [ ]:
samplerate, signal = wavfile.read("cta0001_stereo.wav")
print(signal.shape)

Первое число &mdash; количество отсчётов в сигнале (количество строк в массиве), а второе &mdash; количество каналов (количество столбцов в массиве).

#### Запись файлов

Запишем в новый файл синусоидальный сигнал, но в этот раз сгенерируем его с помощью `numpy`. Зададим параметры:

In [ ]:
import numpy as np

samplerate = 22050
sampwidth = 2
duration = 2  # секунды 
num_samples = samplerate * duration
max_ampl = 2 ** (sampwidth * 8 - (0 if sampwidth == 1 else 1)) - 1
ampl = 0.5 * max_ampl
freq = 100  # герцы

Зададим соответствия между количеством байт на отсчёт и типом данных:

In [ ]:
sampwidth_to_dtype = {
    1: np.uint8,
    2: np.int16,
    4: np.int32,
}

Сгенерируем синусоиду:

In [ ]:
sine = ampl * np.sin(np.arange(num_samples) * freq * 2 * np.pi / samplerate)

Переведём её в нужный тип данных с помощью метода `.astype()`:

In [ ]:
sine_int = sine.astype(sampwidth_to_dtype[sampwidth])

И запишем её в новый файл с помощью функции `write()`, которая принимает на вход имя файла, частоту дискретизации и массив с данными:

In [ ]:
wavfile.write("sine_wave_numpy.wav", samplerate, sine_int)

Ничто не мешает нам использовать и вещественные числа:

In [ ]:
sine_float = sine.astype(np.float32)
wavfile.write("sine_wave_numpy_float.wav", samplerate, sine_float)

#### Чтение файлов без заголовка

Чтобы прочитать файл без заголовка, можно воспользоваться функцией `np.frombuffer()`:

In [ ]:
with open("cta0001.sbl", "rb") as f:
    raw_signal = f.read()

signal = np.frombuffer(raw_signal, dtype=np.int16)

#### Порядок байт в `numpy`

`numpy`, как и `struct`, по умолчанию исползует системный порядок байт. Если вам необходимо указать конкретный порядок байт, определите специальный тип `numpy`:

In [ ]:
dt = np.dtype(np.int16).newbyteorder("<")
signal = np.frombuffer(raw_signal, dtype=dt)

### Работа с файлами WAV с помощью библиотеки `wavio`

Библиотека [`wavio`](https://pypi.org/project/wavio/) также позволяет работать с аудиоданными как с `numpy`-массивами. Она поддерживает 24-битные файлы (но не поддерживает вещественные числа) и позволяет легко переходить от одного формата к другому. В ней определены функции `read()` и `write()`.

#### Чтение файлов

Функция `read()` возвращает специальный объект класса `wavio.Wav`, который содержит три поля: `data` &mdash; `numpy`-массив с данными, `rate` &mdash; частота дискретизации и `sampwidth` &mdash; количество байт на отсчёт.

In [ ]:
import wavio

data = wavio.read("cta0001.wav")
signal, samplerate, sampwidth = data.data, data.rate, data.sampwidth

Массив с данными всегда двумерный, даже когда канал один: в этом случае второе измерение равно единице.

#### Запись файлов

Для записи воспользуемся функцией `wavio.write()`. Функция берёт на вход три обязательных аргумента: файл для записи, данные в виде `numpy`-массива и частота дискретизации. Кроме этого, у неё есть три необязательных параметра: `sampwidth` &mdash; количество байт на отсчёт, `scale` &mdash; коэффициент масштабирования и `clip` &mdash; режим работы с обрезанными данными.

Если не передавать аргумент `sampwidth`, он будет определён на основе типа данных массива. Если значения массива не входят в диапазон типа, соответствующего переданному `sampwidth`, они будут обрезаны (_clipped_). Этот аргумент становится обязательным, если данные не целочисленного типа. Тогда амплитуды масштабируются так, чтобы &minus;1 соответствовало минимальному возможному значению для заданного количества байт на отсчёт, а 1 &mdash; максимальному.

Масштабированием можно управлять и вручную при помощи аргумента `scale`, но только в том случае, если данные вещественного типа. Если это строка `"auto"`, то данные автоматически подстроятся под весь доступный диапазон. Если это число, то данные будут масштабированы так, что амплитуды, равные этому числу, превратятся в максимальное доступное значение.

Наконец, аргумент `clip` определяет, что произойдёт, если при записи значения амплитуды были обрезаны (см. выше). Если он равен строке `"warn"` (по умолчанию), то на экран будет выведено предупреждение, если строке `"raise"` &mdash; будет вызвано исключение, если строке `"ignore"` &mdash; ничего не произойдёт. Запишем нашу синусоиду в файл:

In [ ]:
wavio.write("output_wavio.wav", sine, samplerate, sampwidth=2)

### Работа с файлами WAV с помощью библиотеки `librosa`

[`librosa`](https://librosa.org/doc/latest/index.html) &mdash; это библиотека для анализа музыки и вообще аудио. Она ориентирована в первую очередь на музыку, но содержит и ряд полезных в речевых технологиях инструментов.

В `librosa` аудиосигнал также представлен в виде `numpy`-массива. Чтобы прочитать файл, используем функцию `librosa.load()`. Она берёт на вход в качестве аргументов имя файла и ряд дополнительных параметров (все передаются только по ключевому слову &mdash; _keyword-only_): `sr` &mdash; желаемая частота дискретизации; `mono` &mdash; логическое значение, показывающее, нужно ли переводить сигнал в одноканальную версию; `offset` &mdash; время начала чтения файла в секундах; `duration` &mdash; длительность читаемого фрагмента в секундах; `dtype` &mdash; тип данных выходного массива; `res_type` &mdash; [тип передискретизации](https://librosa.org/doc/latest/generated/librosa.resample.html#librosa.resample).

Обратите внимание, что у аргумента `sr` установлено значение по умолчанию 22050 Гц, и, если его не передавать, сигнал будет автоматически приведён к этой частоте дискретизации. Чтобы не менять частоту дискретизации, передайте в аргумент `sr` `None`.

Функция возвращает два значения: массив с аудиоданными и частоту дискретизации. При этом в массиве все амплитуды будут нормализованы, т.&nbsp;е. приведены в диапазон от &minus;1 до 1 путём деления на максимально возможное значение (в качестве типа данных по умолчанию используется `numpy.float32`).

In [ ]:
import librosa

data, samplerate = librosa.load("cta0001.wav", sr=None)

В аргумент `sr` можно передать любую нужную частоту дискретизации:

In [ ]:
data, samplerate = librosa.load("cta0001.wav", sr=44100)

### Практические задания

#### 1. Деление файла пополам

Напишите программу, которая считывает .wav-файл, делит его на две половины и каждую записывает в отдельный файл. Оформите в виде функции, которая берёт на вход имя файла и возвращает имена получившихся файлов.

**Указания к выполнению**: можно пользоваться любым методом чтения и записи файлов (но помните, что рекомендуемым является чтение с помощью `scipy.io.wavfile`). Использование встроенного модуля `wave` даст дополнительное преимущество: прочитанные данные не нужно преобразовывать в числа из байтового формата, т.&nbsp;к. `bytes` можно поделить на части с помощью срезов, как любые итерируемые объекты. Помните, что длина каждой части должна быть кратна количеству байт на отсчёт!

#### 2. Преобразование из SBL в WAV

Напишите программу, которая считывает .sbl-файл и сохраняет его как .wav. Оформите в виде функции, которая берёт на вход имя файла, частоту дискретизации, количество байт на отсчёт (по умолчанию &mdash; 2) и количество каналов (по умолчанию &mdash; 1) и возвращает имя получившегося файла.

#### 3. Модификация файла WAV

Напишите программу, которая:

1. Считывает моноканальный файл WAV;
2. Делает из него стереофайл так: в правый канал помещает отсчёты левого в обратном порядке;
3. Вставляет паузы 200 мс (или любое другое число, но явно прописанное в коде) на 1/4, 1/2 и 3/4 длительности;
4. Записывает в новый файл.